# Часть первая, с блекджеком и стратегиями

Правила блекджека достаточно просты;
давайте начнём с самой базовой версии, которая реализована в OpenAI Gym:
*  численные значения карт равны от 2 до 10 для карт от двойки до десятки, 10 для валетов, дам и королей;
* туз считается за 11 очков, если общая сумма карт на руке при этом не превосходит 21 (по-английски в этом случае говорят, что на руке есть usable ace), и за 1 очко, если превосходит;
*  игроку раздаются две карты, дилеру — одна в открытую и одна в закрытую;
* игрок может совершать одно из двух действий:
** hit  — взять ещё одну карту;
** stand — не брать больше карт;
* если сумма очков у игрока на руках больше 21, он проигрывает (bust);
* если игрок выбирает stand с суммой не больше 21, дилер добирает карты, пока сумма карт в его руке меньше 17;
* после этого игрок выигрывает, если дилер либо превышает 21, либо получает сумму очков меньше, чем сумма очков у игрока; при равенстве очков объявляется ничья (ставка возвращается);
* в исходных правилах есть ещё дополнительный бонус за natural blackjack: если игрок набирает 21 очко с раздачи, двумя картами, он выигрывает не +1, а +1.5 (полторы ставки).


## Задание 1
Рассмотрим очень простую стратегию: говорить stand, если у нас на руках комбинация в 19, 20 или 21 очко, во всех остальных случаях говорить hit.
Используйте методы Монте-Карло, чтобы оценить выигрыш от этой стратегии.


Card Values:

    Face cards (Jack, Queen, King) have a point value of 10.

    Aces can either count as 11 (called a ‘usable ace’) or 1.

    Numerical cards (2-9) have a value equal to their number.


In [2]:
import gym
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import itertools
from itertools import product
import random
from collections import defaultdict
from tqdm import tqdm
from blackjack import BlackjackEnv
from functools import partialmethod

In [3]:
env = BlackjackEnv()
env.natural = True
type(env)

blackjack.BlackjackEnv

In [12]:
def play_game(env, strategy):
    # sum_hand, dealer, usable_ace
    observation = env.reset()[0]
    terminated = False
    G = 0
    while not terminated:
        # There are two actions: stick (0), and hit (1).
        action = strategy(observation)
        observation, reward, terminated, _, _ = env.step(action)
        G += reward
    return G

def simple_strategy(observation):
    usable_ace = observation[0] + 10 * int(observation[2]) not in (19, 20, 21)
    not_usable_ace = observation[0] not in (19, 20, 21)
    return int(usable_ace or not_usable_ace)


def play_games(env, strategy=simple_strategy, n=100_000):
    rewards = []
    for _ in tqdm(range(n)):
        reward = play_game(env, strategy)
        rewards.append(reward)
    return np.mean(rewards)

In [15]:
play_games(env, strategy=simple_strategy, n=10_000)

100%|██████████| 10000/10000 [00:02<00:00, 4465.82it/s]


-0.17845

Данная стратегия проигрышная, мы в минусе

# Задание 2
Реализуйте метод обучения с подкреплением без модели (можно Q-обучение, но рекомендую попробовать и другие, например Monte Carlo control) для обучения стратегии в блекджеке, используя окружение BlackjackEnv из OpenAI Gym.

In [33]:
S = []
for i in range(1 + 1 + 2, 11 + 11 + 10):
    for j in range(1, 11):
        for k in (True, False):
            if not k or (k and 12 <= i <= 21):
                S.append((i, j, k))
state_index = {state: i for i, state in enumerate(S)}

In [39]:
def get_action_Q(Q):
    return np.argmax(Q, axis=1)

def play_game_Q(env, Q, alpha=0.05, epsilon=0.1, gamma=1):
    observation = env.reset()[0]
    index = state_index[observation]
    terminated = False

    while not terminated:
        action = get_action_Q(Q)[index] if random.random() < (1 - epsilon) else random.choice((0, 1))
        observation, reward, terminated, _, _ = env.step(action)
        index_new = state_index[observation]
        Q[index, action] = Q[index, action] + alpha * (reward + gamma * max(Q[index_new]) - Q[index, action])
        index = index_new
    return Q

def q_learning(env, Q, games=10_000, alpha=0.01, epsilon=0.01, gamma=1):
    for _ in tqdm(range(games)):
        Q = play_game_Q(env=env, Q=Q, alpha=alpha, epsilon=epsilon, gamma=gamma)
    return Q

In [40]:
num_comb = len(list(product(range(1 + 1 + 2, 11 + 11 + 10), range(1, 11), (True, False))))
Q = np.zeros((len(S), 2))
Q = q_learning(env, Q, games=10_000, alpha=0.01, epsilon=0.1)

100%|██████████| 10000/10000 [00:02<00:00, 3706.33it/s]


In [41]:
def Q_strategy(observation):
    return np.argmax(Q, axis=1)[state_index[observation]]

play_games(env, strategy=Q_strategy, n=10_000)

100%|██████████| 10000/10000 [00:02<00:00, 3963.20it/s]


-0.06445

Уже лучше, но всё равно в минусе :(

## Задание 3
Сколько выигрывает казино у вашей стратегии? Нарисуйте графики среднего дохода вашего метода (усреднённого по крайней мере по 100000 раздач, а лучше больше) по ходу обучения. Попробуйте подобрать оптимальные гиперпараметры.

In [ ]:
import optuna

def objective(trial):
    alpha = trial.suggest_float('alpha', 0, 1)
    epsilon = trial.suggest_float('epsilon', 0, 0.1)
    Q = np.zeros((len(S), 2))
    Q = q_learning(env, Q, games=10_000, alpha=alpha, epsilon=epsilon)
    return play_games(env, strategy=Q_strategy, n=10_000)

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=1_000, n_jobs=-1, show_progress_bar=False)

study.best_params

[I 2022-12-30 18:14:17,697] A new study created in memory with name: no-name-41234c7b-bf4f-4418-a88e-a9a4e938e07b
